# Project: Used Car Price Prediction using Linear Regression 
       
- The goal of this project is to predict the used car price based on various features.
- Dataset to downloaded from the below link:
https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset

In [1]:
import time
import numpy as np
import pandas as pd
import re
import math
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder

start = time.time()
# read in the dataset
pd.pandas.set_option('display.max_columns', None)
df = pd.read_csv('used_cars_data.csv')
print("Time taken to read dataset: %f seconds" % (time.time()-start))

df.head()

/Users/jingshu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Time taken to read dataset: 110.044824 seconds


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,522,960,[!@@Additional Info@@!]Engine: 2.4L I4 ZERO EV...,I4,1300.0,I4,Solar Yellow,NaN,NaN,True,Jeep,41.2 in,12.7 gal,Gasoline,NaN,66.5 in,NaN,177.0,Black,NaN,NaN,NaN,True,NaN,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.cargurus.com/images/forsale/202...,['Quick Order Package'],Jeep,5 seats,7.0,Renegade,NaN,"177 hp @ 5,750 RPM",23141.0,NaN,0,2.8,370599.0,Flagship Chrysler,NaN,"200 lb-ft @ 1,750 RPM",A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,207,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,2000.0,I4,Narvik Black,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,Black (Ebony),NaN,NaN,NaN,True,NaN,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.cargurus.com/images/forsale/202...,['Adaptive Cruise Control'],Land Rover,7 seats,8.0,Discovery Sport,NaN,"246 hp @ 5,500 RPM",46500.0,NaN,0,3.0,389227.0,Land Rover San Juan,NaN,"269 lb-ft @ 1,400 RPM",A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,1233,969,NaN,H4,2500.0,H4,None,False,False,True,FIAT,43.3 in,15.9 gal,Gasoline,False,58.1 in,23.0,305.0,None,False,NaN,NaN,False,NaN,18.3467,180.9 in,2017-04-25,UNKNOWN,173473508,-66.1098,NaN,"['Alloy Wheels', 'Bluetooth', 'Backup Camera',...",Subaru,5 seats,NaN,WRX STI,3.0,"305 hp @ 6,000 RPM",46995.0,False,0,NaN,370467.0,FIAT de San Juan,False,"290 lb-ft @ 4,000 RPM",M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,196,922,"[!@@Additional Info@@!]Fog Lights,7 Seat Packa...",V6,3000.0,V6,Eiger Gray,NaN,NaN,True,Land Rover,39 in,23.5 gal,Gasoline,NaN,73 in,NaN,340.0,Gray (Ebony/Ebony/Ebony),NaN,NaN,NaN,True,NaN,18.4439,195.1 in,2020-02-26,GRAY,266911050,-66.0785,https://static.cargurus.com/images/forsale/202...,NaN,Land Rover,7 seats,11.0,Discovery,NaN,"340 hp @ 6,500 RPM",67430.0,NaN,0,3.0,389227.0,Land Rover San Juan,NaN,"332 lb-ft @ 3,500 RPM",A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,137,922,"[!@@Additional Info@@!]Keyless Entry,Ebony Mor...",I4,2000.0,I4,Narvik Black,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,Black (Ebony),NaN,NaN,NaN,True,NaN,18.4439,181 in,2020-04-25,BLACK,270957414,-66.0785,https://static.cargurus.com/images/forsale/202...,['Adaptive Cruise Control'],Land Rover,7 seats,7.0,Discovery Sport,NaN,"246 hp @ 5,500 RPM",48880.0,NaN,0,3.0,389227.0,Land Rover San Juan,NaN,"269 lb-ft @ 1,400 RPM",A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


## Data Preprocessing

### 1- Data initial cleaning
1. **Step-1**: Remove rows with duplicate vehicle id numbers
2. **Step-2**: Remove columns with majority of NaN's
3. **Step-3**: Remove columns according to background knowledge
4. **Step-4**: Process feature "year": (1) drop null rows; (2) convert values of column "year" into a relative number (to 1995) starting from 0.

In [2]:
# percentage of non-NA values
thresh_prcnt_non_na = 0.5

# create drop feature list
feature_drop_list = ['vin', 'power', 'torque', 'engine_cylinders', 'exterior_color', 'interior_color', 'sp_name', 
                     'seller_rating', 'main_picture_url', 'description', 'dealer_zip', 'wheel_system_display', 
                     'trimId', 'trim_name', 'major_options', 'listed_date', 'latitude', 'longitude', 'price']

def initialClean(df):
    ## Step-1: Remove rows with duplicate vehicle id numbers
    print("Original dataset: %d * %d" %(df.shape[0], df.shape[1]))
    df = df.drop_duplicates(subset=["vin"])
    print("Dataset after removing duplicated Ids: %d * %d" %(df.shape[0], df.shape[1]))
    
    ## Step-2: Remove columns with majority of NaN's
    size = len(df)
    thresh = math.floor(size * thresh_prcnt_non_na) # Require that many non-NA values
    df = df.dropna(axis=1, thresh=thresh)
    print("Dataset after removing columns with more than %.1f of NaN's: %d * %d" %((1-thresh_prcnt_non_na), 
                                                                         df.shape[0], df.shape[1]))
    
    price_col = df["price"]
    
    ## Step-3: Remove columns according to background knowledge
    df = df.drop(feature_drop_list, axis=1)
    print("\nNew dataset: %d * %d" %(df.shape[0], df.shape[1]))
    
    ## Step-4: Process feature "year"
    # (1) drop null rows
    df = df.drop(df[df['year'].isnull()==True].index)
    # (2) convert values of column "year" into a relative number
    yrs = df["year"].tolist()
    yrs = list(map(lambda x: x - 1995, yrs))
    yrs = list(map(lambda x: max(x, 0), yrs))
    
    df['year'] = yrs
    
    return df, price_col

In [3]:
df, price_col = initialClean(df)
df.head()

Original dataset: 3000040 * 66
Dataset after removing duplicated Ids: 3000000 * 66
Dataset after removing columns with more than 0.5 of NaN's: 3000000 * 56

New dataset: 3000000 * 37


,back_legroom,body_type,city,city_fuel_economy,daysonmarket,engine_displacement,engine_type,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,isCab,is_new,length,listing_color,listing_id,make_name,maximum_seating,mileage,model_name,salvage,savings_amount,sp_id,theft_title,transmission,transmission_display,wheel_system,wheelbase,width,year
0,35.1 in,SUV / Crossover,Bayamon,NaN,522,1300.0,I4,NaN,NaN,True,Jeep,41.2 in,12.7 gal,Gasoline,NaN,66.5 in,NaN,177.0,NaN,True,166.6 in,YELLOW,237132766,Jeep,5 seats,7.0,Renegade,NaN,0,370599.0,NaN,A,9-Speed Automatic Overdrive,FWD,101.2 in,79.6 in,24
1,38.1 in,SUV / Crossover,San Juan,NaN,207,2000.0,I4,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,NaN,True,181 in,BLACK,265946296,Land Rover,7 seats,8.0,Discovery Sport,NaN,0,389227.0,NaN,A,9-Speed Automatic Overdrive,AWD,107.9 in,85.6 in,25
2,35.4 in,Sedan,Guaynabo,17.0,1233,2500.0,H4,False,False,True,FIAT,43.3 in,15.9 gal,Gasoline,False,58.1 in,23.0,305.0,False,False,180.9 in,UNKNOWN,173473508,Subaru,5 seats,NaN,WRX STI,False,0,370467.0,False,M,6-Speed Manual,AWD,104.3 in,78.9 in,21
3,37.6 in,SUV / Crossover,San Juan,NaN,196,3000.0,V6,NaN,NaN,True,Land Rover,39 in,23.5 gal,Gasoline,NaN,73 in,NaN,340.0,NaN,True,195.1 in,GRAY,266911050,Land Rover,7 seats,11.0,Discovery,NaN,0,389227.0,NaN,A,8-Speed Automatic Overdrive,AWD,115 in,87.4 in,25
4,38.1 in,SUV / Crossover,San Juan,NaN,137,2000.0,I4,NaN,NaN,True,Land Rover,39.1 in,17.7 gal,Gasoline,NaN,68 in,NaN,246.0,NaN,True,181 in,BLACK,270957414,Land Rover,7 seats,7.0,Discovery Sport,NaN,0,389227.0,NaN,A,9-Speed Automatic Overdrive,AWD,107.9 in,85.6 in,25


### 2- Handle features of differing data types
1. **Step-1**: Handle categorical features: (1) extract number from some categorical features (drop measurement unit) and refill the missing value; (2) encode the other categorical features into integers.
2. **Step-2**: Data scaling

In [4]:
## encoder for converting categorical variables: encode and imputation
encoder = Pipeline(steps=[
    ("encoder", OrdinalEncoder()),
    ("imputer", SimpleImputer(strategy="most_frequent"))
])

In [5]:
## scaler for numerical variables: imputation and data scaling
scaler = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler())
])

In [6]:
def handleDifferingFeatures(df):

    for c in df.columns:
#         clear_output(wait=True)
#         print(c)

        item = df[c].dropna(axis=0).tolist()[0]
        
        if type(item) is str:
            ## Extract number from some categorical features (drop measurement unit) and refill the missing value
            if re.search('((\d{1,}.\d{1,}))(in)((--)?)', item) is not None:
                df[c] = convertMeasurementStrings(df, c)
                
            ## Encode the other categorical features into integers
            else:
                arr = np.array(df[c].tolist()).reshape(-1,1)
                df[c] = encoder.fit_transform(arr)
        
        ## imputation and data scaling for numerical features
        arr = np.array(df[c].tolist()).reshape(-1,1)
        df[c] = scaler.fit_transform(arr)
    
    return df

In [7]:
def convertMeasurementStrings(df, label):
    arr = df[label].tolist()
    
    def transform(ele):
        if pd.isnull(ele) or re.search("\d", ele) is None:
            return "0.0"
        return float(ele.split(' ')[0])
    
    arr = list(map(transform, arr))
    arr = [float(x) for x in arr]
    mean = np.mean(arr)
    
    arr = SimpleImputer(strategy="constant", fill_value=mean, 
                        missing_values=0.0).fit_transform(np.array(arr).reshape(-1,1))
    
    return arr

In [8]:
df2 = df.copy(deep=True)
handleDifferingFeatures(df2)
df2.head()

,back_legroom,body_type,city,city_fuel_economy,daysonmarket,engine_displacement,engine_type,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,isCab,is_new,length,listing_color,listing_id,make_name,maximum_seating,mileage,model_name,salvage,savings_amount,sp_id,theft_title,transmission,transmission_display,wheel_system,wheelbase,width,year
0,0.511416,0.555556,0.051430,0.130777,0.145040,0.077922,0.153846,0.212869,0.009502,1.0,0.437500,0.425743,0.115385,0.625,0.154406,0.557203,0.166439,0.128964,0.182364,1.0,0.156699,1.000000,0.828716,0.464646,0.583333,7.000001e-08,0.731092,0.006691,0.0,0.823837,0.003347,0.00,0.886364,0.6,0.024845,0.592982,0.923077
1,0.648402,0.555556,0.781263,0.130777,0.057516,0.168831,0.153846,0.212869,0.009502,1.0,0.500000,0.227723,0.362637,0.625,0.154406,0.588983,0.166439,0.201903,0.182364,1.0,0.324163,0.000000,0.938659,0.535354,0.750000,8.000001e-08,0.298319,0.006691,0.0,0.870482,0.003347,0.00,0.886364,0.4,0.155280,0.792982,0.961538
2,0.525114,0.666667,0.354247,0.083333,0.342595,0.233766,0.000000,0.000000,0.000000,1.0,0.229167,0.633663,0.263736,0.625,0.000000,0.381356,0.111111,0.264271,0.000000,0.0,0.322967,0.857143,0.585812,0.898990,0.583333,3.114691e-04,0.951681,0.000000,0.0,0.823507,0.000000,0.75,0.568182,0.4,0.089027,0.568421,0.807692
3,0.625571,0.555556,0.781263,0.130777,0.054460,0.298701,0.564103,0.212869,0.009502,1.0,0.500000,0.217822,0.631868,0.625,0.154406,0.694915,0.166439,0.301268,0.182364,1.0,0.491627,0.285714,0.942341,0.535354,0.750000,1.100000e-07,0.296919,0.006691,0.0,0.870482,0.003347,0.00,0.772727,0.4,0.293996,0.849123,0.961538
4,0.648402,0.555556,0.781263,0.130777,0.038066,0.168831,0.153846,0.212869,0.009502,1.0,0.500000,0.227723,0.362637,0.625,0.154406,0.588983,0.166439,0.201903,0.182364,1.0,0.324163,0.000000,0.957780,0.535354,0.750000,7.000001e-08,0.298319,0.006691,0.0,0.870482,0.003347,0.00,0.886364,0.4,0.155280,0.792982,0.961538


In [9]:
df3 = pd.concat([price_col, df2], axis=1)
df3.head()

,price,back_legroom,body_type,city,city_fuel_economy,daysonmarket,engine_displacement,engine_type,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,isCab,is_new,length,listing_color,listing_id,make_name,maximum_seating,mileage,model_name,salvage,savings_amount,sp_id,theft_title,transmission,transmission_display,wheel_system,wheelbase,width,year
0,23141.0,0.511416,0.555556,0.051430,0.130777,0.145040,0.077922,0.153846,0.212869,0.009502,1.0,0.437500,0.425743,0.115385,0.625,0.154406,0.557203,0.166439,0.128964,0.182364,1.0,0.156699,1.000000,0.828716,0.464646,0.583333,7.000001e-08,0.731092,0.006691,0.0,0.823837,0.003347,0.00,0.886364,0.6,0.024845,0.592982,0.923077
1,46500.0,0.648402,0.555556,0.781263,0.130777,0.057516,0.168831,0.153846,0.212869,0.009502,1.0,0.500000,0.227723,0.362637,0.625,0.154406,0.588983,0.166439,0.201903,0.182364,1.0,0.324163,0.000000,0.938659,0.535354,0.750000,8.000001e-08,0.298319,0.006691,0.0,0.870482,0.003347,0.00,0.886364,0.4,0.155280,0.792982,0.961538
2,46995.0,0.525114,0.666667,0.354247,0.083333,0.342595,0.233766,0.000000,0.000000,0.000000,1.0,0.229167,0.633663,0.263736,0.625,0.000000,0.381356,0.111111,0.264271,0.000000,0.0,0.322967,0.857143,0.585812,0.898990,0.583333,3.114691e-04,0.951681,0.000000,0.0,0.823507,0.000000,0.75,0.568182,0.4,0.089027,0.568421,0.807692
3,67430.0,0.625571,0.555556,0.781263,0.130777,0.054460,0.298701,0.564103,0.212869,0.009502,1.0,0.500000,0.217822,0.631868,0.625,0.154406,0.694915,0.166439,0.301268,0.182364,1.0,0.491627,0.285714,0.942341,0.535354,0.750000,1.100000e-07,0.296919,0.006691,0.0,0.870482,0.003347,0.00,0.772727,0.4,0.293996,0.849123,0.961538
4,48880.0,0.648402,0.555556,0.781263,0.130777,0.038066,0.168831,0.153846,0.212869,0.009502,1.0,0.500000,0.227723,0.362637,0.625,0.154406,0.588983,0.166439,0.201903,0.182364,1.0,0.324163,0.000000,0.957780,0.535354,0.750000,7.000001e-08,0.298319,0.006691,0.0,0.870482,0.003347,0.00,0.886364,0.4,0.155280,0.792982,0.961538


In [10]:
df3.to_csv('preprocessed_dataset.csv',index=False)